In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from embedding_evaluation_framework import EmbeddingData, test_n2v


In [2]:
inits = 20
num_splits = 100

In [3]:
datasets = ['citeseer','PubMed']

In [4]:
out = 'eval/test_acc_nerd.csv'

In [5]:
def report_test_acc_unsupervised_embedding(cache_prefix,dataset,filename,method,speedup=False,nerd=None):
    tests = []
    for init in range(inits):
        print(f' init {init}')
        emb = EmbeddingData(f'/tmp/{cache_prefix}Emb{dataset.capitalize()}Init{init}',dataset,filename,directed=False,initialization=f'{method}/{init}',nerd=nerd)
        test = test_n2v(emb[0],num_splits=num_splits,speedup=speedup)
        tests = tests + test
    return tests

In [6]:
def model_selection_gnn(df):
    return df[df.val_avg == df.groupby('arch').val_avg.transform(max)]
def model_selection(df):
    return df[df.val_avg == df.val_avg.max()]

In [7]:
if os.path.exists(out):
    test_acc = pd.read_csv(out)
else:
    test_acc = pd.DataFrame(columns='method dataset test_acc test_avg test_std'.split())

In [8]:
test_acc

,method,dataset,test_acc,test_avg,test_std
0,nerd,cora,"[0.7417302798982188, 0.7408821034775234, 0.752...",0.729139,0.016211


In [9]:
df_nerd = pd.read_csv('eval/nerd.csv')

In [10]:
model_selection(df_nerd)

,splits,inits,type,lr,neg,val_acc,val_avg,val_std
20,100,20,hub,0.025,20,"[0.7238095238095238, 0.6904761904761905, 0.752...",0.74519,0.027754
21,100,20,aut,0.025,20,"[0.7238095238095238, 0.6904761904761905, 0.752...",0.74519,0.027754


In [ ]:
lr = 0.025
neg = 20
for dataset in datasets:
    print(dataset)
    tests = report_test_acc_unsupervised_embedding(cache_prefix=f'nerdUndlr{lr}neg{neg}',dataset=dataset,filename=f'line_{neg}_{lr}',method='nerd',
                                                   nerd='aut',speedup=(dataset=='cora_full'))
    test_acc = test_acc.append({'method':'nerd', 'dataset':dataset,'test_acc':tests, 'test_avg':np.mean(tests), 'test_std':np.std(tests)},ignore_index=True)
    test_acc.to_csv(out,index=False)

citeseer
 init 0
Processing...
15
0
Done!
